# Multi-Model LLM Inference Optimization
**Author:** Vaishak Balachandra  
**University:** Purdue University - MS Computer Science  
**Project:** Comprehensive LLM optimization across model sizes

---

## Models:
1. GPT-2 (124M)
2. Mistral-7B (7B)
3. Llama-2-7B (7B) - Optional

## Optimizations:
1. FlashAttention-2
2. 4-bit Quantization
3. FP16 Precision

**Hardware:** A100 GPU  
**Runtime:** ~60 minutes

In [ ]:
# GPU Verification
!nvidia-smi

import torch
print(f"\n{'='*80}")
print("SYSTEM VERIFICATION")
print(f"{'='*80}")
print(f"✅ PyTorch: {torch.__version__}")
print(f"✅ CUDA: {torch.cuda.is_available()}")
print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
print(f"✅ VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

gpu_name = torch.cuda.get_device_name(0)
if any(x in gpu_name for x in ['A100', 'A10', 'H100', 'L4']):
    print(f"\n✅ {gpu_name} supports FlashAttention-2!")
else:
    print(f"\n⚠️ {gpu_name} may not support FlashAttention-2")
print(f"{'='*80}")

## HuggingFace Authentication (Optional - for Llama-2)

In [ ]:
from huggingface_hub import login
from getpass import getpass

print("🔐 HuggingFace Authentication (for Llama-2)")
print("="*80)

use_llama = input("Include Llama-2? (y/n): ").lower().strip()

LLAMA_AVAILABLE = False

if use_llama == 'y':
    try:
        hf_token = getpass("Paste your HF token (hidden): ")
        login(token=hf_token, add_to_git_credential=False)

        # Verify access
        from huggingface_hub import list_repo_files
        files = list_repo_files("meta-llama/Llama-2-7b-hf", token=hf_token)

        print("\n✅ Authenticated! Llama-2 will be included")
        LLAMA_AVAILABLE = True

        # Store token in environment
        import os
        os.environ['HF_TOKEN'] = hf_token
        os.environ['HUGGING_FACE_HUB_TOKEN'] = hf_token

    except Exception as e:
        print(f"\n❌ Authentication failed: {e}")
        print("Continuing without Llama-2")
else:
    print("\n⚠️ Skipping Llama-2")

print(f"\n📊 Models: GPT-2, Mistral-7B" + (", Llama-2-7B" if LLAMA_AVAILABLE else ""))

In [ ]:
# Install packages
print("📦 Installing packages (5-8 minutes)...\n")

!pip install -q transformers accelerate bitsandbytes sentencepiece protobuf
!pip install -q flash-attn --no-build-isolation

print("\n✅ Installation complete!")

import transformers
import accelerate
import bitsandbytes

print(f"✅ transformers: {transformers.__version__}")
print(f"✅ accelerate: {accelerate.__version__}")
print(f"✅ bitsandbytes: {bitsandbytes.__version__}")

In [ ]:
# Benchmarking Framework
import numpy as np
import time
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

ALL_RESULTS = []

def benchmark_model(model, tokenizer, model_name, config_name,
                   prompt="Explain quantum computing:",
                   num_runs=8, max_new_tokens=100):
    device = next(model.parameters()).device
    times = []

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    input_len = inputs.input_ids.shape[1]

    print(f"\n{'='*80}")
    print(f"🔬 BENCHMARKING: {model_name} - {config_name}")
    print(f"{'='*80}")
    print(f"📝 Prompt: '{prompt[:60]}...'")
    print(f"🎯 Runs: {num_runs} + 1 warmup\n")

    # Warmup
    print("🔥 Warmup...", end=" ")
    try:
        with torch.no_grad():
            _ = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id if tokenizer.eos_token_id else tokenizer.pad_token_id
            )
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        print("✅")
    except Exception as e:
        print(f"❌ Failed: {e}")
        return None

    # Benchmark
    for i in range(num_runs):
        try:
            if torch.cuda.is_available():
                torch.cuda.synchronize()

            start = time.time()
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=max_new_tokens,
                    do_sample=False,
                    pad_token_id=tokenizer.eos_token_id if tokenizer.eos_token_id else tokenizer.pad_token_id
                )

            if torch.cuda.is_available():
                torch.cuda.synchronize()
            end = time.time()

            elapsed = end - start
            tokens_gen = outputs.shape[1] - input_len
            times.append(elapsed)

            print(f"📊 Run {i+1:2d}: {elapsed:.3f}s ({tokens_gen} tok, {tokens_gen/elapsed:.1f} tok/s)")
        except Exception as e:
            print(f"❌ Run {i+1} failed: {e}")
            continue

    if len(times) == 0:
        return None

    times = np.array(times)
    throughputs = max_new_tokens / times

    try:
        memory_gb = model.get_memory_footprint() / 1e9
    except:
        memory_gb = torch.cuda.memory_allocated() / 1e9

    results = {
        'model': model_name,
        'config': config_name,
        'mean_time': float(np.mean(times)),
        'std_time': float(np.std(times)),
        'median_throughput': float(np.median(throughputs)),
        'mean_throughput': float(np.mean(throughputs)),
        'std_throughput': float(np.std(throughputs)),
        'memory_gb': memory_gb,
        'params_b': model.num_parameters() / 1e9 if hasattr(model, 'num_parameters') else 0
    }

    print(f"\n📊 RESULTS:")
    print(f"   ⏱️  {results['mean_time']:.3f}s ± {results['std_time']:.3f}s")
    print(f"   🚀 {results['median_throughput']:.1f} tok/s")
    print(f"   💾 {results['memory_gb']:.2f} GB")
    print(f"{'='*80}")

    ALL_RESULTS.append(results)
    return results

def clear_memory():
    import gc
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    print("🗑️ Memory cleared")

print("✅ Benchmark framework ready")

---
## MODEL 1: GPT-2 (124M)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

print("\n" + "="*80)
print("MODEL 1: GPT-2 (124M)")
print("="*80)

gpt2_id = "gpt2"
tokenizer_gpt2 = AutoTokenizer.from_pretrained(gpt2_id)
tokenizer_gpt2.pad_token = tokenizer_gpt2.eos_token

In [ ]:
# GPT-2 Baseline (FP32)
print("\n1️⃣ GPT-2 Baseline (FP32)")
model_gpt2_base = AutoModelForCausalLM.from_pretrained(gpt2_id).to("cuda")
results_gpt2_base = benchmark_model(model_gpt2_base, tokenizer_gpt2, "GPT-2", "Baseline (FP32)")
del model_gpt2_base
clear_memory()

In [ ]:
# GPT-2 FP16
print("\n2️⃣ GPT-2 FP16")
model_gpt2_fp16 = AutoModelForCausalLM.from_pretrained(
    gpt2_id,
    torch_dtype=torch.float16
).to("cuda")
results_gpt2_fp16 = benchmark_model(model_gpt2_fp16, tokenizer_gpt2, "GPT-2", "FP16")
del model_gpt2_fp16
clear_memory()

In [ ]:
# GPT-2 INT8
print("\n3️⃣ GPT-2 INT8")
model_gpt2_int8 = AutoModelForCausalLM.from_pretrained(
    gpt2_id,
    device_map="auto",
    load_in_8bit=True
)
results_gpt2_int8 = benchmark_model(model_gpt2_int8, tokenizer_gpt2, "GPT-2", "INT8")
del model_gpt2_int8
clear_memory()

---
## MODEL 2: MISTRAL-7B (7B)

In [ ]:
print("\n" + "="*80)
print("MODEL 2: MISTRAL-7B (7B)")
print("="*80)

mistral_id = "mistralai/Mistral-7B-v0.1"
tokenizer_mistral = AutoTokenizer.from_pretrained(mistral_id)

print("\n📥 Loading Mistral-7B (first time: ~5 min)...")

In [ ]:
# Mistral Baseline
print("\n1️⃣ Mistral-7B Baseline (FP16)")
model_mistral_base = AutoModelForCausalLM.from_pretrained(
    mistral_id,
    torch_dtype=torch.float16,
    device_map="auto"
)
results_mistral_base = benchmark_model(model_mistral_base, tokenizer_mistral, "Mistral-7B", "Baseline (FP16)")
mistral_baseline = results_mistral_base['median_throughput']
del model_mistral_base
clear_memory()

In [ ]:
# Mistral FlashAttention-2
print("\n2️⃣ Mistral-7B FlashAttention-2")
try:
    model_mistral_flash = AutoModelForCausalLM.from_pretrained(
        mistral_id,
        torch_dtype=torch.float16,
        device_map="auto",
        attn_implementation="flash_attention_2"
    )
    results_mistral_flash = benchmark_model(model_mistral_flash, tokenizer_mistral, "Mistral-7B", "FlashAttention-2")
    del model_mistral_flash
except Exception as e:
    print(f"❌ FlashAttention failed: {e}")
clear_memory()

In [ ]:
# Mistral 4-bit
from transformers import BitsAndBytesConfig

print("\n3️⃣ Mistral-7B 4-bit")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
model_mistral_4bit = AutoModelForCausalLM.from_pretrained(
    mistral_id,
    quantization_config=bnb_config,
    device_map="auto"
)
results_mistral_4bit = benchmark_model(model_mistral_4bit, tokenizer_mistral, "Mistral-7B", "4-bit (NF4)")
del model_mistral_4bit
clear_memory()

---
## MODEL 3: LLAMA-2-7B (7B) - Optional

In [ ]:
if LLAMA_AVAILABLE:
    print("\n" + "="*80)
    print("MODEL 3: LLAMA-2-7B (7B)")
    print("="*80)

    llama_id = "meta-llama/Llama-2-7b-hf"

    # Get token from environment
    import os
    from huggingface_hub import HfFolder

    hf_token = os.environ.get('HF_TOKEN') or HfFolder.get_token()

    if hf_token:
        print(f"🔐 Token: {hf_token[:15]}...")
    else:
        print("❌ No token!")

    # Load tokenizer
    tokenizer_llama = AutoTokenizer.from_pretrained(llama_id, token=hf_token)
    tokenizer_llama.pad_token = tokenizer_llama.eos_token

    print("✅ Llama-2 tokenizer loaded")
else:
    print("\n⚠️ Skipping Llama-2")

In [ ]:
if LLAMA_AVAILABLE:
    # Llama Baseline
    import os
    hf_token = os.environ.get('HF_TOKEN')

    print("\n1️⃣ Llama-2-7B Baseline (FP16)")
    model_llama_base = AutoModelForCausalLM.from_pretrained(
        llama_id,
        torch_dtype=torch.float16,
        device_map="auto",
        token=hf_token
    )
    results_llama_base = benchmark_model(model_llama_base, tokenizer_llama, "Llama-2-7B", "Baseline (FP16)")
    del model_llama_base
    clear_memory()

In [ ]:
if LLAMA_AVAILABLE:
    # Llama FlashAttention
    import os
    hf_token = os.environ.get('HF_TOKEN')

    print("\n2️⃣ Llama-2-7B FlashAttention-2")
    try:
        model_llama_flash = AutoModelForCausalLM.from_pretrained(
            llama_id,
            torch_dtype=torch.float16,
            device_map="auto",
            attn_implementation="flash_attention_2",
            token=hf_token
        )
        results_llama_flash = benchmark_model(model_llama_flash, tokenizer_llama, "Llama-2-7B", "FlashAttention-2")
        del model_llama_flash
    except Exception as e:
        print(f"❌ FlashAttention failed: {e}")
    clear_memory()

In [ ]:
if LLAMA_AVAILABLE:
    # Llama 4-bit
    import os
    hf_token = os.environ.get('HF_TOKEN')

    print("\n3️⃣ Llama-2-7B 4-bit")
    model_llama_4bit = AutoModelForCausalLM.from_pretrained(
        llama_id,
        quantization_config=bnb_config,
        device_map="auto",
        token=hf_token
    )
    results_llama_4bit = benchmark_model(model_llama_4bit, tokenizer_llama, "Llama-2-7B", "4-bit (NF4)")
    del model_llama_4bit
    clear_memory()

---
## Results Analysis

In [ ]:
# Results table
df = pd.DataFrame(ALL_RESULTS)

print("\n" + "="*100)
print("🎯 COMPLETE RESULTS")
print("="*100)
print()
print(df[['model', 'config', 'median_throughput', 'memory_gb']].to_string(index=False))
print("\n" + "="*100)

# Save CSV
df.to_csv('/content/benchmark_results.csv', index=False)
print("\n✅ Saved: /content/benchmark_results.csv")

In [ ]:
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Throughput
pivot = df.pivot(index='config', columns='model', values='median_throughput')
pivot.plot(kind='bar', ax=axes[0], width=0.8)
axes[0].set_title('Throughput Comparison', fontsize=16, fontweight='bold')
axes[0].set_ylabel('Tokens/Second', fontsize=13)
axes[0].grid(axis='y', alpha=0.3)
plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=45, ha='right')

# Memory
pivot_mem = df.pivot(index='config', columns='model', values='memory_gb')
pivot_mem.plot(kind='bar', ax=axes[1], width=0.8)
axes[1].set_title('Memory Usage', fontsize=16, fontweight='bold')
axes[1].set_ylabel('GPU Memory (GB)', fontsize=13)
axes[1].grid(axis='y', alpha=0.3)
plt.setp(axes[1].xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.savefig('/content/optimization_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: /content/optimization_results.png")

In [ ]:
# Final Summary
print("\n" + "="*80)
print("📝 RESUME BULLETS")
print("="*80)

mistral_flash = df[(df['model'] == 'Mistral-7B') & (df['config'] == 'FlashAttention-2')]
mistral_base = df[(df['model'] == 'Mistral-7B') & (df['config'].str.contains('Baseline'))]

if len(mistral_flash) > 0 and len(mistral_base) > 0:
    speedup = mistral_flash.iloc[0]['median_throughput'] / mistral_base.iloc[0]['median_throughput']
    print(f"\n• Accelerated Mistral-7B inference by {speedup:.1f}x through FlashAttention-2")
    print(f"  CUDA optimization, reducing memory bandwidth bottlenecks")

print(f"\n• Conducted multi-model optimization study across {df['model'].nunique()} models")
print(f"  with {len(df)} configurations, demonstrating optimization scaling")

print("\n" + "="*80)
print("✅ PROJECT COMPLETE!")
print("="*80)

In [ ]:
import shutil
from google.colab import files

# Create a zip with all files
!mkdir -p /content/project_files
!cp /content/benchmark_results.csv /content/project_files/
!cp /content/optimization_results.png /content/project_files/

# Download the notebook
# (You'll need to manually download the .ipynb file: File → Download → Download .ipynb)

# Zip everything
!cd /content && zip -r llm_optimization_project.zip project_files/

print("✅ Files packaged!")
print("📦 Downloading zip file...")

files.download('/content/llm_optimization_project.zip')